# Heros of Pymoli <a id='top'></a>

The data of purchansing items is provided. We generate tables to analyze the data and make observations from the tables.

### 1. Load data

* [Load data](#load_data) purchase_data.csv 
    
### 2. Analysis

1. [Player Count](#player_count)

2. [Purchasing Analysis (Total)](#Purchasing_Analysis_Total)

3. [Gender Demographics](#Gender_Demographics)

4. [Purchasing Analysis (Gender)](#Purchasing_Analysis_Gender)

5. [Age Demographics](#Age_Demographics)

6. [Purchasing Analysis (Age group)](#Purchasing_Analysis_Age)

7. [Top Spenders](#Top_Spenders)

8. [Most Popular Items](#Most_Popular_Items)

9. [Most Profitable Items](#Most_Profitable_Items)

### 3. Trends

* [Ovsevations in purchasing items](#trends)

In [56]:
# Dependencies and Setup
import pandas as pd

## Load data 
<a id='load_data'></a> ([To Top](#top))

In [57]:
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_df = pd.read_csv(file_to_load)
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [58]:
purchase_df.unique()

Purchase ID    780
SN             576
Age             39
Gender           3
Item ID        179
Item Name      179
Price          145
dtype: int64

In [100]:
purchase_df.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,91.755128,3.050987
std,225.310896,6.659444,52.697702,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,47.750000,1.980000
50%,389.500000,22.000000,92.000000,3.150000
75%,584.250000,25.000000,138.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


In [59]:
purchase_df = purchase_df.convert_dtypes()
purchase_df.dtypes

Purchase ID      Int64
SN              string
Age              Int64
Gender          string
Item ID          Int64
Item Name       string
Price          float64
dtype: object

## Analysis

In [101]:
#-----------------------------------
# Display_DF(df)
#    returns a modified dataframe with cleaner formatting
#    such as '$' or '%' or float format. 
#-----------------------------------
def Display_DF(df):
    
    df2 = df.copy()
    currency_ch = ['Price', 'Revenue', 'Value']
    for col in df2.columns:
        
        if any(ext in col for ext in currency_ch):
            df2[col]=df[col].map('${:,.2f}'.format)

        elif "Percentage" in col:
            df2[col]=df[col].map('{:,.2f}%'.format)

        elif df[col].dtype==float:
            df2[col]=df[col].map('{:,.2f}'.format)
            
    return df2

### 1. Player Count<a id='player_count'></a>

* Display the total number of players [(To Top)](#top)

In [61]:
# number of players 
total_players = purchase_df['SN'].nunique()

# Print the dataframe
pd.DataFrame({'Total Players': [total_players]})

,Total Players
0,576


### 2. Purchasing Analysis (Total)<a id='Purchasing_Analysis_Total'></a>

* Display the number of items, the average price of the items, the number of purchases and total revenue [(To Top)](#top)

In [62]:
# Set the summary(total) dataframe
total_df = pd.DataFrame({
    'Number of Unique Items' : purchase_df['Item ID'].nunique(),
    'Average Price' : purchase_df['Price'].mean(),
    'Number of Purchases' : purchase_df['Purchase ID'].nunique(),
    'Total Revenue' : purchase_df['Price'].sum()},
    index=[0])

# Print the dataframe
Display_DF(total_df)

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


### 3. Gender Demographics<a id='Gender_Demographics'></a>

* Display the percentage and the total count of Male/Female/Other players [(To Top)](#top)

In [63]:
# Group purchase data by 'Gender'
gender_gp = purchase_df.groupby(['Gender'])

# Calculate the total players of each gender group
players_count = gender_gp['SN'].nunique()

# Set the demographic summary dataframe
gender_demo = pd.DataFrame({
        'Total Count' : players_count,
        'Percentage of Players' : 100*(players_count/total_players)})

# Print the dataframe
Display_DF(gender_demo.sort_values(by=['Total Count'],ascending=False))

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


### 4. Purchasing Analysis (Gender) <a id='Purchasing_Analysis_Gender'></a>

* Display the purchase count, the average purchase price, total purchase value and the average total purchase per person of Male/Female/Other players [(To Top)](#top)

In [71]:
# Create the summary(purchase) dataframe from the gender groups data
perc_purchase = 100*(gender_gp['Item ID'].count()/float(total_df['Number of Purchases']))
gender_purchase = pd.DataFrame({
    'Purchase Count' : gender_gp['Item ID'].count(),
    'Percentage Purchase' : perc_purchase,
    'Average Purchase Price' : gender_gp['Price'].mean(),
    'Total Purchase Value' : gender_gp['Price'].sum(),
    'Avg Total Purchase per Person' : gender_gp['Price'].sum().div(gender_demo['Total Count'])
})

# Print the dataframe
Display_DF(gender_purchase)

,Purchase Count,Percentage Purchase,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,,
Female,113,14.49%,$3.20,$361.94,4.47
Male,652,83.59%,$3.02,"$1,967.64",4.07
Other / Non-Disclosed,15,1.92%,$3.35,$50.19,4.56


### 5. Age Demographics<a id='Age_Demographics'></a>

* Display the percentage and the total count of each age group [(To Top)](#top)

In [49]:
# Set bins as the age ranges
bins      = [0,      10,      15,      20,      25,      30,      35,      40, 100]
bins_name = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# Add the column 'Age Ranges' by binning
purchase_df['Age Ranges'] = pd.cut(purchase_df['Age'], bins, right=False, labels=bins_name,)

# Group purchase data by 'Age Ranges'
age_gp = purchase_df.groupby(['Age Ranges'])

# Calculate the total players of each age group
players_count = age_gp['SN'].nunique()

# Set the demographic summary dataframe
age_demo = pd.DataFrame({
        'Total Count' : players_count,
        'Percentage of Players' : 100*(players_count/total_players)})

# Print the dataframe
Display_DF(age_demo)

,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


### 6. Purchasing Analysis (Age)<a id='Purchasing_Analysis_Age'></a>

* Display the purchase count, the average purchase price, total purchase value and the average total purchase per person of each age group [(To Top)](#top)

In [50]:
# Create the summary(purchase) dataframe about the age groups
age_purchase = pd.DataFrame({
    'Purchase Count' : age_gp['Item ID'].count(),
    'Average Purchase Price' : age_gp['Price'].mean(),
    'Total Purchase Value' : age_gp['Price'].sum(),
    'Avg Total Purchase per Person' : age_gp['Price'].sum().div(age_demo['Total Count'])
})

# Print the dataframe
Display_DF(age_purchase)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,4.54
10-14,28,$2.96,$82.78,3.76
15-19,136,$3.04,$412.89,3.86
20-24,365,$3.05,"$1,114.06",4.32
25-29,101,$2.90,$293.00,3.81
30-34,73,$2.93,$214.00,4.12
35-39,41,$3.60,$147.67,4.76
40+,13,$2.94,$38.24,3.19


### 7. Top Spenders<a id='Top_Spenders'></a>

* Display the top five players with the purchase count, the average purchase price and total purchase value [(To Top)](#top)

In [79]:
# Group the purchase data by players (SN)
player_gp = purchase_df.groupby('SN')

# Create the summary(purchase) dataframe about each player
player_purchase = pd.DataFrame({
    'Purchase Count' : player_gp['Item ID'].count(),
    'Average Purchase Price' : player_gp['Price'].mean(),
    'Total Purchase Value' : player_gp['Price'].sum()})

# Print the dataframe
Display_DF(player_purchase.sort_values(by=['Total Purchase Value'], ascending=False)).head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


### 8. Most Popular Items<a id='Most_Popular_Items'></a>

* Checking the purchase count, display the top five items with their name, purchase count, item price, total purchase value [(To Top)](#top)

In [102]:
# Group the purchase data by items
item_gp = purchase_df.groupby(['Item ID','Item Name'])

# Create the summary(purchase) dataframe about each item
item_purchase = pd.DataFrame({
    'Purchase Count' : item_gp['Item ID'].count(),
    'Item Price' : item_gp['Price'].mean(),
    'Total Purchase Value' : item_gp['Price'].sum()})

# Print the purchase summary about the items
# from the most purchased item
Display_DF(item_purchase.sort_values(by=['Purchase Count'], ascending=False)).head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


### 9. Most Profitable Items<a id='Most_Profitable_Items'></a>

* Checking the total purchase value, display the top five items with their name, purchase count, item price, total purchase value [(To Top)](#top)

In [97]:
# Print the purchase summary about the items
# from the highest total purchased value
Display_DF(item_purchase.sort_values(by=['Total Purchase Value'], ascending=False)).head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


## Trends<a id='trends'></a>

* The game is popular among men than women with high percent differece, 84% vs. 14%. ([Table](#Gender_Demographics))

* The game is popular most among the people in the age group 20~24. ([Table](#Age_Demographics))

* The players normally spend about 4 dollars to buy an item. ([Table-Gender](#Purchasing_Analysis_Gender), [Table-Age](#Purchasing_Analysis_Age))

* The items, Final Critic and Oathbreaker-Last Hope of the Breaking Storm, are most popular and most profitable. ([Table-Count](#Most_Popular_Items), [Table-Profit](#Most_Profitable_Items))

([To Top](#top))